In [18]:
import os
import tempfile
from typing import Tuple, Dict, List

from common import *
import xml.etree.ElementTree as ET


In [36]:
def load_workflow(path: str) -> Graph:
    graph = get_graph()
    graph.parse(path, format='turtle')
    return graph

In [37]:
def get_workflow_steps(graph: Graph) -> list:
    steps = list(graph.subjects(RDF.type, dtbox.Step))
    return steps

In [38]:
def get_base_config():
    root = ET.Element('config', {'xmlns': 'http://www.knime.org/2008/09/XMLConfig',
                                 'xmlns:xsi': 'http://www.w3.org/2001/XMLSchema-instance',
                                 'xsi:schemaLocation': 'http://www.knime.org/2008/09/XMLConfig http://www.knime.org/XMLConfig_2008_09.xsd',
                                 'key': 'settings.xml',
                                 })

    ET.SubElement(root, 'entry', {'key': 'node_file', 'type': 'xstring', 'value': 'settings.xml'})
    ET.SubElement(root, 'entry', {'key': 'customDescription', 'type': 'xstring', 'isnull': 'true', 'value': ''})
    ET.SubElement(root, 'entry', {'key': 'state', 'type': 'xstring', 'value': 'CONFIGURED'})
    ET.SubElement(root, 'entry', {'key': 'hasContent', 'type': 'xboolean', 'value': 'false'})
    ET.SubElement(root, 'entry', {'key': 'isInactive', 'type': 'xboolean', 'value': 'false'})
    ET.SubElement(root, 'config', {'key': 'factory_settings'})
    ET.SubElement(root, 'entry', {'key': 'node-feature-name', 'type': 'xstring', 'isnull': 'true', 'value': ''})
    ET.SubElement(root, 'entry',
                  {'key': 'node-feature-symbolic-name', 'type': 'xstring', 'isnull': 'true', 'value': ''})
    ET.SubElement(root, 'entry', {'key': 'node-feature-vendor', 'type': 'xstring', 'isnull': 'true', 'value': ''})
    ET.SubElement(root, 'entry', {'key': 'node-feature-version', 'type': 'xstring', 'value': '0.0.0'})

    return ET.ElementTree(root)

In [39]:
def get_step_component_implementation(ontology: Graph, workflow_graph: Graph, step: URIRef) -> Tuple[URIRef, URIRef]:
    component = next(workflow_graph.objects(step, dtbox.runs, True))
    implementation = next(ontology.objects(component, dtbox.hasImplementation, True))
    return component, implementation

In [46]:
def get_knime_properties(ontology: Graph, implementation: URIRef) -> Dict[str, str]:
    results = {}
    for p, o in ontology.predicate_objects(implementation):
        if p.fragment.startswith('knime'):
            results[p.fragment[6:]] =  o.value
    return results

In [50]:
def create_step_file(ontology: Graph, workflow_graph:Graph, step:URIRef, folder, iterator: int) -> None:
    tree = get_base_config()
    root = tree.getroot()
    component, implementation = get_step_component_implementation(ontology, workflow_graph, step)
    properties = get_knime_properties(ontology, implementation)

    for key, value in properties.items():
        ET.SubElement(root, 'entry', {'key': key, 'type': 'xstring', 'value': value})

    path_name = properties["node-name"].replace('(', '_').replace(')', '_')
    subfolder = os.path.join(folder, f'{path_name} (#{iterator})')
    os.mkdir(subfolder)
    tree.write(os.path.join(subfolder, 'settings.xml'), encoding='utf-8', xml_declaration=True)


In [32]:
ontology = get_ontology_graph()
source_path = '../pipeline_generator/workflows/2023-07-15 21-40-38/workflow_0_DescriptionIntent_b949aa3c_e664_43eb_ae32_004cac74fd24.ttl'
destination_path = './workflows/2023-07-15 20-27-10/workflow_10_DescriptionIntent_4762afff_2235_4c9e_b3ac_6876f5b634ed.knwf'

temp_folder = tempfile.mkdtemp()
print(temp_folder)

C:\Users\Victor\AppData\Local\Temp\tmp7r8scph2


In [33]:
graph = load_workflow(source_path)
steps = get_workflow_steps(graph)
print(steps)

[rdflib.term.URIRef('https://diviloper.dev/ontology/Workflow#workflow_0_DescriptionIntent_b949aa3c_e664_43eb_ae32_004cac74fd24-step_0_component_random_absolute_train_test_split'), rdflib.term.URIRef('https://diviloper.dev/ontology/Workflow#workflow_0_DescriptionIntent_b949aa3c_e664_43eb_ae32_004cac74fd24-step_1_component_decimal_scaling'), rdflib.term.URIRef('https://diviloper.dev/ontology/Workflow#workflow_0_DescriptionIntent_b949aa3c_e664_43eb_ae32_004cac74fd24-step_3_component_drop_rows_with_missing_values'), rdflib.term.URIRef('https://diviloper.dev/ontology/Workflow#workflow_0_DescriptionIntent_b949aa3c_e664_43eb_ae32_004cac74fd24-step_5_component_hypertangent_svm_learner'), rdflib.term.URIRef('https://diviloper.dev/ontology/Workflow#workflow_0_DescriptionIntent_b949aa3c_e664_43eb_ae32_004cac74fd24-step_2_component_normalizer_applier'), rdflib.term.URIRef('https://diviloper.dev/ontology/Workflow#workflow_0_DescriptionIntent_b949aa3c_e664_43eb_ae32_004cac74fd24-step_4_component_mis

In [51]:
create_step_file(ontology, graph, steps[0], temp_folder, 0)

In [29]:
def translate_graph_folder(ontology, source_folder, destination_folder):
    workflows = [f for f in os.listdir(source_folder) if f.endswith('.ttl')]
    for workflow in workflows:
        source_path = os.path.join(source_folder, workflow)
        destination_path = os.path.join(destination_folder, workflow[:-4] + '.knwf')
        translate_graph(ontology, source_path, destination_path)